# Visualization: Trading Session

In [7]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [8]:
from trading_bot.agent import Agent

model_name = 'cut_50'
test_stock = 'data/2019cut.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [9]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


### 3. Running Eval

In [10]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

# Set logging level to DEBUG
logging.basicConfig(level=logging.DEBUG)

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]
test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)


2024-03-15 13:41:31 DESKTOP-RSIH8GO root[13976] DEBUG switching to TensorFlow for CPU
2024-03-15 13:41:31 DESKTOP-RSIH8GO root[13976] DEBUG Buy at: 1045.85VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Sell at: 1016.06VND | Position: -29.79VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Buy at: 1070.33VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Buy at: 1057.19VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Buy at: 1044.69VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Sell at: 1098.26VND | Position: +27.93VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Sell at: 1075.57VND | Position: +18.38VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Sell at: 1073.90VND | Position: +29.21VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] DEBUG Buy at: 1060.62VND
2024-03-15 13:41:32 DESKTOP-RSIH8GO root[13976] INFO cut_50: +45.73VND



In [11]:
history

[(1045.849976, 'BUY'),
 (1016.0599980000001, 'SELL'),
 (1070.709961, 'HOLD'),
 (1068.390015, 'HOLD'),
 (1076.280029, 'HOLD'),
 (1074.660034, 'HOLD'),
 (1070.329956, 'BUY'),
 (1057.189941, 'BUY'),
 (1044.689941, 'BUY'),
 (1077.150024, 'HOLD'),
 (1080.969971, 'HOLD'),
 (1089.900024, 'HOLD'),
 (1098.26001, 'SELL'),
 (1070.52002, 'HOLD'),
 (1075.569946, 'SELL'),
 (1073.900024, 'SELL'),
 (1090.98999, 'HOLD'),
 (1070.079956, 'HOLD'),
 (1060.619995, 'BUY')]

### 4. Visualize

In [12]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='.2f', title='Price (VND)'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='.2f', title='Price (VND)'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [13]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [62]:
df.head()

,date,actual,position,action
0,2019-01-02,1045.849976,1045.849976,HOLD
1,2019-01-03,1016.059998,1045.849976,BUY
2,2019-01-04,1070.709961,1016.059998,SELL
3,2019-01-07,1068.390015,1070.709961,HOLD
4,2019-01-08,1076.280029,1068.390015,HOLD


In [61]:
import pandas as pd
import os

# Đường dẫn đến thư mục
directory = r'D:\Test1\trading-bot-master\extra'

# Đảm bảo rằng đường dẫn tồn tại
if not os.path.exists(directory):
    os.makedirs(directory)

# Lưu DataFrame thành file CSV
df.to_csv(os.path.join(directory, f'{model_name}_result.csv'), index=False)
